# Лабораторна робота №3

## Тема

Вступ до опрацювання природної мови

## Мета

здобуття студентом навичок реалізації базових методів обробки
природної мови, включно з попереднім опрацюванням тексту, формуванням «мішка
слів» («bag-of-words»), виділенням стоп-слів і найважливіших слів у документі,
створенням тематичних моделей.

## Автор

Молчанов Михайло, ІА-12

## Виконання

### Завантаження та зчитування тексту

У цьому блоці коду ми реалізуємо завантаження англійської книги *"Alice's Adventures in Wonderland"* із проекту Gutenberg. Основні дії включають:

- **Завантаження тексту** за URL за допомогою бібліотеки `requests`.
- **Збереження тексту** у локальному файлі `alice_in_wonderland.txt` з використанням кодування `utf-8`.
- **Зчитування тексту** з файлу у змінну `text` для подальшого опрацювання.


In [11]:
# Завантаження книги
import requests

url = "http://www.gutenberg.org/files/11/11-0.txt"
response = requests.get(url)
with open("data/alice_in_wonderland.txt", "w", encoding="utf-8") as file:
    file.write(response.text)

# Зчитування тексту
with open("data/alice_in_wonderland.txt", "r", encoding="utf-8") as file:
    text = file.read()


Результат виконання:

- Текст книги успішно завантажено з інтернету та збережено в локальному файлі.
- Тепер повний текст доступний у змінній `text`, і ми можемо почати його обробку.


### Попередня обробка тексту

На цьому етапі виконується попереднє опрацювання тексту, яке включає:

1. **Імпорт бібліотек:**
   - `nltk` для токенізації та роботи зі стоп-словами.
   - `re` для роботи з регулярними виразами.
2. **Завантаження необхідних даних:**
   - Завантажуються стоп-слова та токенізатор із бібліотеки `nltk`.
3. **Реалізація функції `preprocess_text`:**
   - Приведення тексту до нижнього регістру.
   - Видалення цифр за допомогою регулярного виразу.
   - Видалення пунктуації та неалфавітних символів.
   - Токенізація тексту, тобто розбиття на окремі слова.
   - Видалення стоп-слів і неалфавітних токенів.


In [12]:
# Попередня обробка тексту
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # до нижнього регістру
    text = re.sub(r'\d+', '', text)  # видалення цифр
    text = re.sub(r'[^\w\s]', '', text)  # видалення пунктуації
    tokens = word_tokenize(text)  # токенізація
    filtered_tokens = [word for word in tokens if word not in stop_words and word.isalpha()]  # видалення стоп-слів
    return filtered_tokens

processed_text = preprocess_text(text)


[nltk_data] Downloading package punkt to /Users/killer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/killer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Результат виконання:

- Текст очищено від зайвих символів, цифр і стоп-слів. Він представлений у вигляді списку токенів, які готові до подальшого аналізу.
- `processed_text` містить попередньо оброблений текст книги, що є зручним форматом для роботи з алгоритмами NLP.


### Розділення тексту на глави

У цьому блоці коду текст книги розділяється на окремі глави для подальшого аналізу. Основні кроки:

1. **Розділення тексту:**
   - Використовується метод `split("CHAPTER")` для поділу тексту на частини на основі ключового слова "CHAPTER".
   - Пропускається перший елемент (`[1:]`), щоб уникнути тексту перед першою главою.
2. **Відновлення заголовків:**
   - До кожної частини тексту додається слово "CHAPTER", щоб зберегти формат глав.
3. **Попередня обробка:**
   - Кожна глава обробляється за допомогою функції `preprocess_text`, описаної раніше, що очищує текст від зайвих символів, цифр та стоп-слів.


In [13]:
# Розділення на глави
chapters = text.split("CHAPTER")[1:]  # Розділення тексту по главам
chapters = ["CHAPTER" + chapter for chapter in chapters]  # Додавання заголовка назад
preprocessed_chapters = [preprocess_text(chapter) for chapter in chapters]


Результат виконання:

- Текст книги успішно розділено на глави, а також кожна глава очищена та представлена у вигляді списку токенів.
- Змінна `preprocessed_chapters` містить список оброблених глав, готових до аналізу за алгоритмами TF-IDF та LDA.


### Визначення топ-20 слів для кожної глави за TF-IDF

У цьому блоці коду виконується обчислення топ-20 ключових слів для кожної глави книги на основі алгоритму TF-IDF (Term Frequency-Inverse Document Frequency). Основні кроки:

1. **Імпорт необхідних бібліотек:**
   - Використовується `TfidfVectorizer` з `scikit-learn`.
2. **Ініціалізація векторизатора:**
   - Параметр `max_features=20` обмежує кількість ключових слів до 20 для кожної глави.
3. **Обчислення TF-IDF:**
   - Текст кожної глави об'єднується в одну строку.
   - `fit_transform` обчислює ваги TF-IDF для кожного слова.
   - За допомогою `get_feature_names_out()` отримуються ключові слова, а з матриці TF-IDF витягуються їхні відповідні значення.
4. **Сортування результатів:**
   - Слова сортуються за вагами TF-IDF у порядку спадання.


### Як працює TF-IDF

1. **Term Frequency (TF):**
   - Оцінює частоту появи слова у тексті глави. Чим частіше слово зустрічається, тим більший TF.
   $
   TF(t) = \frac{\text{Кількість появ слова } t \text{ у главі}}{\text{Загальна кількість слів у главі}}
   $
2. **Inverse Document Frequency (IDF):**
   - Зменшує важливість часто вживаних слів (на кшталт "the", "and"), які зустрічаються у багатьох документах.
   $
   IDF(t) = \log{\frac{\text{Загальна кількість глав}}{1 + \text{Кількість глав, у яких зустрічається } t}}
   $
3. **Розрахунок TF-IDF:**
   - Вага TF-IDF для слова обчислюється як добуток TF та IDF.
   $
   TF\text{-}IDF(t) = TF(t) \times IDF(t)
   $


In [14]:
# Визначення топ-20 слів для кожної глави за TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=20)
tfidf_results = []

for chapter in preprocessed_chapters:
    chapter_text = " ".join(chapter)
    tfidf_matrix = tfidf_vectorizer.fit_transform([chapter_text])
    tfidf_terms = tfidf_vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray()[0]
    tfidf_results.append(sorted(zip(tfidf_terms, tfidf_scores), key=lambda x: x[1], reverse=True))

tfidf_results


[[('chapter', 0.7071067811865475), ('rabbithole', 0.7071067811865475)],
 [('chapter', 0.5), ('ii', 0.5), ('pool', 0.5), ('tears', 0.5)],
 [('caucusrace', 0.4472135954999579),
  ('chapter', 0.4472135954999579),
  ('iii', 0.4472135954999579),
  ('long', 0.4472135954999579),
  ('tale', 0.4472135954999579)],
 [('bill', 0.4082482904638631),
  ('chapter', 0.4082482904638631),
  ('iv', 0.4082482904638631),
  ('little', 0.4082482904638631),
  ('rabbit', 0.4082482904638631),
  ('sends', 0.4082482904638631)],
 [('advice', 0.5773502691896258),
  ('caterpillar', 0.5773502691896258),
  ('chapter', 0.5773502691896258)],
 [('chapter', 0.5), ('pepper', 0.5), ('pig', 0.5), ('vi', 0.5)],
 [('chapter', 0.5), ('mad', 0.5), ('teaparty', 0.5), ('vii', 0.5)],
 [('chapter', 0.5), ('croquetground', 0.5), ('queens', 0.5), ('viii', 0.5)],
 [('chapter', 0.4472135954999579),
  ('ix', 0.4472135954999579),
  ('mock', 0.4472135954999579),
  ('story', 0.4472135954999579),
  ('turtles', 0.4472135954999579)],
 [('chapte

### Результат виконання

- Для кожної глави визначено топ-20 ключових слів із відповідними вагами TF-IDF.
- Наприклад, у першій главі слова "chapter" і "rabbithole" мають найбільшу вагу, що вказує на їхню значущість у контексті цієї глави.

TF-IDF дозволяє ідентифікувати унікальні слова для кожного розділу, відфільтровуючи загальні терміни. Це ефективний метод для аналізу тексту та виявлення ключових слів.


### Реалізація LDA (Latent Dirichlet Allocation)
У цьому блоці коду реалізується тематичне моделювання тексту книги за допомогою алгоритму **LDA** (Latent Dirichlet Allocation). Основні кроки:

1. **Імпорт бібліотек:**
   - `LatentDirichletAllocation` для реалізації алгоритму LDA.
   - `CountVectorizer` для перетворення тексту в числову матрицю частот слів.

2. **Векторизація тексту:**
   - Використовується `CountVectorizer` для побудови матриці частот слів.
   - Параметр `max_features=1000` обмежує кількість унікальних слів, які враховуються, до 1000.
   - Текст усіх глав об'єднується в один список, де кожна глава — це окремий документ.

3. **Навчання LDA:**
   - Алгоритм LDA налаштовується на знаходження 5 тем (`n_components=5`) у тексті.
   - `random_state=42` забезпечує відтворюваність результатів.
   - `lda.fit` навчає модель на матриці частот слів.

4. **Отримання топ-слів для кожної теми:**
   - `lda.components_` повертає ваги слів для кожної теми.
   - Найважливіші слова для кожної теми сортуються за їхніми вагами та відбирається топ-20 слів.

### Як працює LDA:
1. **Ідея алгоритму:**
   - LDA — це алгоритм, що припускає, що кожен документ (глава) є сумішшю різних тем, а кожна тема — це набір слів із певними ймовірностями.
   - Наприклад, у тексті книги можуть бути теми, пов'язані з персонажами, місцями або подіями.

2. **Розподіл тем:**
   - LDA визначає, які теми найкраще описують кожну главу, і які слова характерні для цих тем.

3. **Побудова тем:**
   - Кожне слово в тексті отримує вагу в кожній темі. Топ-слова з найвищими вагами визначають характер теми.


In [15]:
# Реалізація LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000, stop_words='english')
lda_matrix = vectorizer.fit_transform([" ".join(chapter) for chapter in preprocessed_chapters])

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(lda_matrix)

lda_topics = lda.components_
lda_terms = vectorizer.get_feature_names_out()

# Отримання топ-слів для кожної теми
lda_results = []
for topic_idx, topic in enumerate(lda_topics):
    lda_results.append(sorted(zip(lda_terms, topic), key=lambda x: x[1], reverse=True)[:20])

lda_results


[[('alice', 44.2826729486036),
  ('little', 41.81616526203385),
  ('rabbit', 17.610056170840295),
  ('thought', 17.173069278184247),
  ('said', 16.032227421457005),
  ('like', 15.115756211127685),
  ('oh', 14.191846970110726),
  ('sure', 14.074789348830507),
  ('door', 13.268383941349947),
  ('moment', 13.014584779230438),
  ('way', 12.796301840215294),
  ('quite', 12.69138537382185),
  ('went', 12.56874231989401),
  ('came', 12.487646314394004),
  ('wonder', 11.501106785710798),
  ('heard', 11.281867911916308),
  ('im', 10.689982437206343),
  ('thing', 10.506302943519906),
  ('feet', 10.197633292998859),
  ('fan', 10.19707560495302)],
 [('said', 115.62880593656809),
  ('alice', 97.43771248039693),
  ('hatter', 53.19293736342241),
  ('dormouse', 39.19599694001947),
  ('mouse', 35.020109164234846),
  ('know', 32.20895191123559),
  ('march', 28.181248113307365),
  ('hare', 27.181326282971057),
  ('time', 24.457723817277486),
  ('little', 22.534506741095853),
  ('went', 19.932960827782317


### Результат виконання:
- Для кожної з 5 тем алгоритм визначив топ-20 слів.
- Наприклад, у першій темі слова "alice", "little", "rabbit" мають найбільші ваги, що вказує на те, що ця тема пов'язана з головним персонажем і її взаємодією з кроликом.
- Тематичне моделювання дає змогу зрозуміти основні ідеї та контекст тексту.

## Висновок

На цій лабораторній роботі я здобув навички реалізації базових методів обробки
природної мови, включно з попереднім опрацюванням тексту, формуванням «мішка
слів» («bag-of-words»), виділенням стоп-слів і найважливіших слів у документі,
створив тематичні моделі.